<a href="https://colab.research.google.com/github/mavela/text-classification-with-svm/blob/main/text_classification_long.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Let's get data from Github first

In [1]:
! git clone https://github.com/mavela/text-classification-with-svm.git


Cloning into 'text-classification-with-svm'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 30 (delta 10), reused 22 (delta 5), pack-reused 0
Unpacking objects: 100% (30/30), done.


In [2]:
%cd text-classification-with-svm/
! ls

/content/text-classification-with-svm
analyze.py   na_train.txt  op_train.txt		   README.md  svm_scripts.py
na_test.txt  op_test.txt   pb_smallpart.conllu.gz  svm.py


## Let's see how it looks like - print 20 first lines

In [3]:
! zcat pb_smallpart.conllu.gz | head -20

# <doc id="7-510353" length="0-1k" crawl_date="2015-06-05" url="http://yle.fi/uutiset/lahden_paikallisliikenteen_uudistus_edennyt_ilman_suuria_ongelmia/7338226?origin=rss" langdiff="0.11">
# delex_lm_mean_perplexity: 210.51
# lex_lm_mean_perplexity: 32064.49
# predicted register: narrative
# <p heading="0">
# paragraph_delex_lm_mean_perplexity: 368.98
# paragraph_lexical_lm_mean_perplexity: 50604.32
# text = Suurilta ja toistuvilta myöhästymisiltä tai muilta kommelluksilta on vältytty.
# </p>
1	Suurilta	suuri	ADJ	_	Case=Abl|Degree=Pos|Number=Plur	4	amod	_	_
2	ja	ja	CCONJ	_	_	3	cc	_	_
3	toistuvilta	toistuva	ADJ	_	Case=Abl|Degree=Pos|Number=Plur	1	conj	_	_
4	myöhästymisiltä	myöhästyminen	NOUN	_	Case=Nom|Derivation=Minen|Number=Plur	9	obl	_	_
5	tai	tai	CCONJ	_	_	7	cc	_	_
6	muilta	muu	PRON	_	Case=Abl|Number=Sing|PronType=Dem	7	det	_	_
7	kommelluksilta	kommellus	NOUN	_	Case=Abl|Number=Plur	4	conj	_	_
8	on	olla	AUX	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	9	aux:pass	

### Fetch register data from the parsebank file

First we must extract our text classes from the parsebank file. The following classes are available:

* how-to/instructions
* informational description
* informational persuasion general
* interactive discussion
* machine translated
* narrative
* opinion

In [4]:
from analyze import extract_register

extract_register("opinion", "pb_smallpart.conllu.gz")

extract_register("narrative", "pb_smallpart.conllu.gz")

Wrote opinion texts to a file!
Wrote narrative texts to a file!


<_io.TextIOWrapper name='narrative_ext.conllu' mode='w' encoding='UTF-8'>

In [5]:
!head -20 narrative_ext.conllu # if you want to have a look at the data, run this!

# predicted register: narrative

1	Suurilta	suuri	ADJ	_	Case=Abl|Degree=Pos|Number=Plur	4	amod	_	_
2	ja	ja	CCONJ	_	_	3	cc	_	_
3	toistuvilta	toistuva	ADJ	_	Case=Abl|Degree=Pos|Number=Plur	1	conj	_	_
4	myöhästymisiltä	myöhästyminen	NOUN	_	Case=Nom|Derivation=Minen|Number=Plur	9	obl	_	_
5	tai	tai	CCONJ	_	_	7	cc	_	_
6	muilta	muu	PRON	_	Case=Abl|Number=Sing|PronType=Dem	7	det	_	_
7	kommelluksilta	kommellus	NOUN	_	Case=Abl|Number=Plur	4	conj	_	_
8	on	olla	AUX	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	9	aux:pass	_	_
9	vältytty	välttyä	VERB	_	Case=Nom|Degree=Pos|Number=Sing|PartForm=Past|VerbForm=Part|Voice=Pass	0	root	_	_
10	.	.	PUNCT	_	_	9	punct	_	_

1	–	–	PUNCT	_	_	5	punct	_	_
2	Tietooni	tieto	NOUN	_	Case=Ill|Number=Sing|Number[psor]=Sing|Person[psor]=1	5	obl	_	_
3	ei	ei	AUX	_	Number=Sing|Person=3|Polarity=Neg|VerbForm=Fin|Voice=Act	5	aux	_	_
4	ole	olla	AUX	_	Connegative=Yes|Mood=Ind|Tense=Pres|VerbForm=Fin	5	aux	_	_
5	tullut	tulla	VERB	_	Case=Nom|Degree=Pos|Number=S

## Then let's save just the texts

In [6]:
from svm_scripts import save_text_format

save_text_format("narrative_ext.conllu","FORM") # nevermind the "FORM", we'll get to that later
save_text_format("opinion_ext.conllu", "FORM")



In [7]:
! wc -l narrative_ext.conllu_out.txt # here we print how many lines the file has. oups, a bit of an imbalance! 
! wc -l opinion_ext.conllu_out.txt

1583 narrative_ext.conllu_out.txt
155 opinion_ext.conllu_out.txt


### Then let's divide the data to train and text, and give register labels to the texts

In [8]:
! cat narrative_ext.conllu_out.txt | head -100 | perl -pe 's/^/NA\t/g' > na_train.txt
! cat opinion_ext.conllu_out.txt | head -100 | perl -pe 's/^/OP\t/g' >  op_train.txt

! cat narrative_ext.conllu_out.txt | head -155 | tail -55 | perl -pe 's/^/NA\t/g' > na_test.txt
! cat opinion_ext.conllu_out.txt | head -155 | tail -55 | perl -pe 's/^/OP\t/g' > op_test.txt

! cat op_test.txt | head -3

OP	perjantai 4. heinäkuuta 2014 112 - vihaan itseäni Linnea Parkkosen ( s. 1995 ) kolmas kirja 112 – vihaan itseäni sijoittuu nuorten maailmaan , mutta joka on terveellistä myös vanhempien lukea , sillä kirja kertoo salakavalasta sairaudesta anoreksiasta . Kirjan päähenkilö on yhdeksäsluokkalainen Lilian , kiltti koululainen , joka opiskeli ahkerasti , harrasti ja jolta ei puuttunut ystäviä . Vanhempi sisko oli muuttanut jo kotoa pois , mutta Lilian eli Lilli vietti paljon aikaa hänen kanssaan . Jopa niin paljon , että unohti välillä kotiintuloaikansa . Leijona seisoi nyt jo niin lähellä , että sen hengityksen tunsi kuonollaan . - Et sä siihen kykene . Niin kuin et yleensä mihinkään , mitä sulta niin harvoin odotetaan . hemmoteltu lapsi . Saat kaiken etkä kuitenkaan mitään arvosta . Sulle kannetaan ruoka pöytään pitkien työpäivien jälkeen ja ootat vielä , että me täällä iltaisin sun tuloos asti jaksetaan valvoa . Huomenna on arkipäivä ja sullakin on koulua ! Se on sun elämässäs tärkeem

In [9]:
! cat na_train.txt op_train.txt | shuf  > train.txt
! cat na_test.txt op_test.txt | shuf > test.txt

## Then finally the classification!


In [10]:
! python3 svm.py train.txt test.txt

              precision    recall  f1-score   support

          NA       0.99      0.99      0.99       100
          OP       0.99      0.99      0.99       100

    accuracy                           0.99       200
   macro avg       0.99      0.99      0.99       200
weighted avg       0.99      0.99      0.99       200

